In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [5]:
# Bring in data
df = pd.read_csv('modeling_dfs/users_order_by_prev_dept.csv')

In [34]:
display(df)

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,...,"('department', 'household')","('department', 'international')","('department', 'meat seafood')","('department', 'missing')","('department', 'other')","('department', 'pantry')","('department', 'personal care')","('department', 'pets')","('department', 'produce')","('department', 'snacks')"
0,0,1187899,1,train,11,4,8,14.0,1.011,1.010,...,0,0,0,0,0,0,0,0,0,3
1,1,1492625,2,train,15,1,11,30.0,2.015,2.014,...,0,1,0,0,0,1,0,0,4,1
2,2,2196797,5,train,5,0,11,6.0,5.005,5.004,...,0,1,0,0,0,0,0,0,5,0
3,3,525192,7,train,21,2,11,6.0,7.021,7.020,...,0,0,1,0,0,0,0,0,4,1
4,4,880375,8,train,4,1,14,10.0,8.004,8.003,...,0,0,0,0,0,1,0,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,131204,2585586,206199,train,20,2,16,30.0,206199.020,206199.019,...,4,0,0,2,0,1,0,0,1,9
131205,131205,943915,206200,train,24,6,19,6.0,206200.024,206200.023,...,0,0,1,0,0,6,0,0,16,1
131206,131206,2371631,206203,train,6,4,19,30.0,206203.006,206203.005,...,1,0,0,0,0,1,0,0,1,7
131207,131207,1716008,206205,train,4,1,16,10.0,206205.004,206205.003,...,0,0,0,0,0,0,0,0,3,0


In [53]:
# all parameters not specified are set to their defaults
feature_cols = ['order_number','order_dow','order_hour_of_day','days_since_prior_order']

feature_cols += [colname for colname in df.columns.values if 'department' in colname] # do not include departments
print('Features:')
print(feature_cols)

logisticRegr = LogisticRegression(penalty='l1',solver='liblinear')
logisticRegr.fit(df[feature_cols],df['any_organic'])

Features:
['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', "('department', 'alcohol')", "('department', 'babies')", "('department', 'bakery')", "('department', 'beverages')", "('department', 'breakfast')", "('department', 'bulk')", "('department', 'canned goods')", "('department', 'dairy eggs')", "('department', 'deli')", "('department', 'dry goods pasta')", "('department', 'frozen')", "('department', 'household')", "('department', 'international')", "('department', 'meat seafood')", "('department', 'missing')", "('department', 'other')", "('department', 'pantry')", "('department', 'personal care')", "('department', 'pets')", "('department', 'produce')", "('department', 'snacks')"]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
predictions = logisticRegr.predict(df[feature_cols])
df['predicted_any_organic'] = predictions
display(df[['predicted_any_organic','any_organic']])
display(df['predicted_any_organic'].value_counts())
display(df['any_organic'].value_counts())

,predicted_any_organic,any_organic
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
131204,True,True
131205,True,True
131206,True,True
131207,True,True


True     114632
False     16577
Name: predicted_any_organic, dtype: int64

True     93287
False    37922
Name: any_organic, dtype: int64

In [56]:
# Use score method to get accuracy of model
score = logisticRegr.score(df[feature_cols], df['any_organic'])
print(score)

cm = metrics.confusion_matrix(df['any_organic'], df['predicted_any_organic'])
print(cm)
#                       Predicted not-organic,0   Predicted organic,1
# True not-organic,0
# True organic,1

0.7408790555525917
[[10250 27672]
 [ 6327 86960]]


In [78]:
coefs = pd.DataFrame(list(zip(logisticRegr.coef_[0], feature_cols)),columns=['coef','feature'])
display(coefs.sort_values('coef'))

# coef_dict = {}
# for coef, feat in zip(logisticRegr.coef_,feature_cols):
#     coef_dict[feat] = coef
# display(coef_dict)

,coef,feature
4,-0.404760,"('department', 'alcohol')"
22,-0.249413,"('department', 'pets')"
19,-0.218548,"('department', 'other')"
15,-0.196764,"('department', 'household')"
6,-0.082840,"('department', 'bakery')"
8,-0.078267,"('department', 'breakfast')"
17,-0.073321,"('department', 'meat seafood')"
21,-0.054953,"('department', 'personal care')"
7,-0.054177,"('department', 'beverages')"
20,-0.036883,"('department', 'pantry')"
